In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')


********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [4]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2831) (12128, 2831)


In [5]:
features_list =[
#Bureau Intutive features
'Num_Enq_all_loans_last_9M_3M',
'Utilization_all',
'total_emi',
'Utilization_CREDIT CARD_opened_last_36M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'num_CREDIT CARD_enq_last_6M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'highest_sanction_amount_credit_cards',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M',
'Age_of_oldest_open_account',
'num_CONSUMER LOAN_enq_last_6M_36M',
    
#Banking Intutive features
'num_credit_gt_salary',    
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M1',    
'Min_Balance_Min_M1_M2_M3',
'ratio_max_credit_to_total_credit_M1',
'spend_vs_salary_ratio',
'bounced_transactions_count',
'ratio_total_debit_to_max_credit_M2_M3',
'min_balance_last_2_months',
'days_balance_lt_5000_M2',
'days_balance_lt_10000_M3',
'num_credits_M2',
'num_debits_M3',
'balance_3th_day_M1',
    
'Max_Credit_Avg_M1_M2_M3',
'Max_Credit_Diff_M1_M2_M3',
'amount_debits_last_2_months',
'sum_insurance_debit',
'avg_num_debits_last_1_months',
'upi_trans_debit_sum',
'avg_num_credits_last_3_months',
'invest_spending_pct',
'cash_withdrawal_sum',

    
#ID Features,

'g406s',
'dm206s',

# Reconsalited Features

'percentage_ever_delinquent', 
'percentage_utilization_gt_75',
'Num_of_installment_loan_opened_last_6M',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'month_year',
'90_in_24mob'
              
]




In [6]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3022161/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3022161/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 42), (15604, 42), (23406,), (15604,))

In [9]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 42) (4682, 42) (15604, 42)


In [10]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [11]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
     'min_child_weight':25,
    'lambda': 5.5,
    'alpha':5.0,
    'gamma':4.5,
    'scale_pos_weight':4.0,
    'random_state': 24
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 600,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.60795	eval-auc:0.61009
[10]	train-auc:0.64203	eval-auc:0.64775
[20]	train-auc:0.64482	eval-auc:0.64700
[30]	train-auc:0.64785	eval-auc:0.64822
[40]	train-auc:0.64924	eval-auc:0.64906
[50]	train-auc:0.64992	eval-auc:0.64917
[60]	train-auc:0.65195	eval-auc:0.64982
[70]	train-auc:0.65488	eval-auc:0.65158
[80]	train-auc:0.65617	eval-auc:0.65235
[90]	train-auc:0.65664	eval-auc:0.65273
[100]	train-auc:0.65810	eval-auc:0.65380
[110]	train-auc:0.65908	eval-auc:0.65400
[120]	train-auc:0.65997	eval-auc:0.65443
[130]	train-auc:0.66088	eval-auc:0.65515
[140]	train-auc:0.66189	eval-auc:0.65599
[150]	train-auc:0.66254	eval-auc:0.65642
[160]	train-auc:0.66396	eval-auc:0.65752
[170]	train-auc:0.66539	eval-auc:0.65893
[180]	train-auc:0.66632	eval-auc:0.65936
[190]	train-auc:0.66709	eval-auc:0.65970
[200]	train-auc:0.66815	eval-auc:0.66059
[210]	train-auc:0.66935	eval-auc:0.66135
[220]	train-auc:0.67007	eval-auc:0.66169
[230]	train-auc:0.67114	eval-auc:0.66261
[240]	train-auc:0.67214	eva

In [13]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.38, KS: 0.27
Eval AUC: 0.67, Gini: 0.34, KS: 0.26
Test AUC: 0.67, Gini: 0.34, KS: 0.24
OOT AUC: 0.68, Gini: 0.35, KS: 0.26


In [14]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.398292         0.656486        0.834476        0.591436   1873   
9          0.259615         0.555449        0.591375        0.524525   1872   
8          0.203526         0.499279        0.524524        0.475426   1872   
7          0.180459         0.454563        0.475424        0.435202   1873   
6          0.169872         0.418930        0.435189        0.403351   1872   
5          0.135684         0.388191        0.403324        0.373802   1872   
4          0.129738         0.361334        0.373791        0.348764   1873   
3          0.092415         0.334784        0.348735        0.320131   1872   
2          0.068910         0.306917        0.320112        0.289041   1872   
1          0.043780         0.258647        0.289030        0.168480   1873   

        label_good  label_bad  
Decile                         
10            1127        746  
9             1386        486  
8             1491        381  
7             1535        338  
6             1554        318  
5             1618        254  
4             1630        243  
3             1699        173  
2             1743        129  
1             1791         82

In [15]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.368994         0.655782        0.827167        0.589487   1561   
9          0.265385         0.553609        0.589413        0.521701   1560   
8          0.214744         0.496212        0.521690        0.473186   1560   
7          0.178091         0.453024        0.473177        0.433916   1561   
6          0.161538         0.417227        0.433898        0.401194   1560   
5          0.150000         0.386578        0.401153        0.373382   1560   
4          0.114670         0.360948        0.373352        0.348800   1561   
3          0.106410         0.334610        0.348796        0.320745   1560   
2          0.085897         0.306686        0.320730        0.288854   1560   
1          0.063421         0.257532        0.288784        0.172233   1561   

        label_good  label_bad  
Decile                         
10             985        576  
9             1146        414  
8             1225        335  
7             1283        278  
6             1308        252  
5             1326        234  
4             1382        179  
3             1394        166  
2             1426        134  
1             1462         99

In [16]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.364386         0.656632        0.823415        0.592999   1213   
9          0.278648         0.555599        0.592984        0.524711   1213   
8          0.215169         0.499697        0.524700        0.475703   1213   
7          0.188944         0.455570        0.475673        0.436671   1212   
6          0.165705         0.419164        0.436658        0.403387   1213   
5          0.154988         0.388234        0.403383        0.373890   1213   
4          0.115512         0.361621        0.373876        0.349359   1212   
3          0.101401         0.335733        0.349349        0.322226   1213   
2          0.077494         0.309410        0.322198        0.293838   1213   
1          0.060181         0.261958        0.293832        0.172064   1213   

        label_good  label_bad  
Decile                         
10             771        442  
9              875        338  
8              952        261  
7              983        229  
6             1012        201  
5             1025        188  
4             1072        140  
3             1090        123  
2             1119         94  
1             1140         73

In [17]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
35                                             g406s       170.0
7               highest_sanction_amount_credit_cards       156.0
10                        Age_of_oldest_open_account       139.0
0                       Num_Enq_all_loans_last_9M_3M       118.0
34                               cash_withdrawal_sum       104.0
37                        percentage_ever_delinquent        90.0
1                                    Utilization_all        84.0
4           Utilization_PERSONAL LOAN_opened_last_9M        83.0
41                         Num_Enq_non_BANK_last_12M        73.0
21                           days_balance_lt_5000_M2        63.0
11                 num_CONSUMER LOAN_enq_last_6M_36M        62.0
3            Utilization_CREDIT CARD_opened_last_36M        61.0
40                          No of enq in past 90days        55.0
18                        bounced_transactions_count        55.0
5                        

In [18]:
feature_importance_df

Feature  Importance
35                                             g406s       170.0
7               highest_sanction_amount_credit_cards       156.0
10                        Age_of_oldest_open_account       139.0
0                       Num_Enq_all_loans_last_9M_3M       118.0
34                               cash_withdrawal_sum       104.0
37                        percentage_ever_delinquent        90.0
1                                    Utilization_all        84.0
4           Utilization_PERSONAL LOAN_opened_last_9M        83.0
41                         Num_Enq_non_BANK_last_12M        73.0
21                           days_balance_lt_5000_M2        63.0
11                 num_CONSUMER LOAN_enq_last_6M_36M        62.0
3            Utilization_CREDIT CARD_opened_last_36M        61.0
40                          No of enq in past 90days        55.0
18                        bounced_transactions_count        55.0
5                        num_CREDIT CARD_enq_last_6M        53.0
6             Num_of_PERSONAL LOAN_opened_last6M_12M        42.0
13                                   upi_trans_count        38.0
29                               sum_insurance_debit        29.0
2                                          total_emi        28.0
20                         min_balance_last_2_months        26.0
17                             spend_vs_salary_ratio        24.0
39            Num_of_installment_loan_opened_last_6M        23.0
15                          Min_Balance_Min_M1_M2_M3        22.0
8                 num_PL_LE_1_Lac_opened_in_last_12M        21.0
30                      avg_num_debits_last_1_months        21.0
22                          days_balance_lt_10000_M3        19.0
36                                            dm206s        17.0
12                              num_credit_gt_salary        16.0
26                           Max_Credit_Avg_M1_M2_M3        13.0
38                      percentage_utilization_gt_75        11.0
33                               invest_spending_pct        10.0
9   Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M         9.0
16               ratio_max_credit_to_total_credit_M1         9.0
14          ratio_balance_after_7_days_max_credit_M1         8.0
31                               upi_trans_debit_sum         8.0
19             ratio_total_debit_to_max_credit_M2_M3         7.0
25                                balance_3th_day_M1         6.0
32                     avg_num_credits_last_3_months         6.0
27                          Max_Credit_Diff_M1_M2_M3         5.0
23                                    num_credits_M2         4.0
24                                     num_debits_M3         4.0
28                       amount_debits_last_2_months         3.0

In [19]:
#Save the pred 
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBI_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_BBI_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_BBI_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_BBI_Oot_Pred_9024.csv',index=False)


In [20]:
# Save the model
xgb_model.save_model('LeoPayu_BBI_90_in_24mob.json')
print('Model Saved')

Model Saved
